In [1]:
import itertools
import os

os.environ["DGLBACKEND"] = "pytorch"

import dgl
import dgl.data
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F

d:\miniconda\envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [3]:
'''
训练集边数：测试集边数 = 9:1
正样本：所有实际存在的边中随机选取90%作为训练集，剩下的10%作为测试集
负样本：所有不存在的边中随机采样与正样本训练集/测试集相同数量的边
'''
u, v = g.edges()

eids = np.arange(g.num_edges())
eids = np.random.permutation(eids) #NOTE: shuffle

test_size = int(len(eids) * 0.1)
train_size = g.num_edges() - test_size

'''
从边中选取正样本边的起点和终点
'''

test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

# Find all negative edges and split them for training and testing
'''
产生负样本，即不存在的边
随机选取负样本边的起点和终点
'''
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.num_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.num_edges())
test_neg_u, test_neg_v = (
    neg_u[neg_eids[:test_size]],
    neg_v[neg_eids[:test_size]],
)
train_neg_u, train_neg_v = (
    neg_u[neg_eids[test_size:]],
    neg_v[neg_eids[test_size:]],
)

# NOTE: 从训练的图中移除测试集的边
train_g = dgl.remove_edges(g,eids[:test_size])

## 任务描述：
各节点通过图神经网络训练学习一个嵌入表示$h_{v}$, 然后训练一个函数$f(h_{v},h_{u})$来预测节点$u,v$之间存在边的概率。

In [4]:
'''
节点学习一个嵌入表示
'''

from dgl.nn import SAGEConv


# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, "mean")
        self.conv2 = SAGEConv(h_feats, h_feats, "mean")

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

## 预测边是否存在的概率需要一个节点对，这个节点对可以看作是原图中的一条边，这条边的属性是由两端节点的属性计算得到。
- 1. 在Train_Graph中训练图神经网络，得到节点的嵌入表示$h_{v}$。
- 2. *计算* Train_Positive_Graph & Train_Negative_Graph中的边的嵌入表示, 获得损失函数。
- 3. *测试* Test_Graph中的边的嵌入表示, 预测边是否存在。

In [5]:
'''
构建训练图和测试图
'''

train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.num_nodes()) # 正图
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.num_nodes()) # 负图

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.num_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.num_nodes())

In [6]:
'''
用上面创建的图来计算边上的节点特征
第1种方法: 将两端的节点特征点乘获得一个scalar特征
- 先把特征赋值到图上
- 用apply_edges函数计算边上的特征
'''

import dgl.function as fn


class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v("h", "h", "score"))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata["score"][:, 0]

In [7]:
'''
第2种方法: 将两端的节点特征拼接后通过一个MLP获得一个scalar特征
- 先把特征赋值到图上
- 用apply_edges函数计算边上的特征
'''

class MLPPredictor(nn.Module):
    '''
    Input: 所有节点的特征h
    Output: 所有边的特征score
    
    '''
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src["h"], edges.dst["h"]], 1)
        return {"score": self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        '''
        先给图赋值，再应用apply_edges函数对所有边进行计算
        '''
        with g.local_scope():
            g.ndata["h"] = h
            g.apply_edges(self.apply_edges)
            return g.edata["score"]

In [ ]:
'''
实例化模型和预测器，并计算损失
'''

model = GraphSAGE(train_g.ndata["feat"].shape[1], 16)
# You can replace DotPredictor with MLPPredictor.
pred = MLPPredictor(16)
# pred = DotPredictor()


def compute_loss(pos_score, neg_score):
    # 正负样本及对应的标签concat，计算二分类交叉熵。
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    ) # 正样本标签为1，负样本标签为0
    return F.binary_cross_entropy_with_logits(scores, labels)

#NOTE: 注意需要import sklearn中的roc_auc_score
from sklearn.metrics import roc_auc_score

def compute_auc(pos_score, neg_score):
    # 计算正负样本的roc_auc_score
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    ).numpy()
    return roc_auc_score(labels, scores)

In [9]:
# ----------- 3. set up loss and optimizer -------------- #
# in this case, loss will in training loop
optimizer = torch.optim.Adam(
    itertools.chain(model.parameters(), pred.parameters()), lr=0.01
) #NOTE: 同时训练时避免使用多个优化器

# ----------- 4. training -------------------------------- #
'''
训练流程：
- 在train_g上计算节点特征
- 在train_pos_g和train_neg_g上计算边的Score
- 计算loss
- Backward and optimize

'''
all_logits = []
for e in range(100):
    # forward
    h = model(train_g, train_g.ndata["feat"])
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print("In epoch {}, loss: {}".format(e, loss))

# ----------- 5. check results ------------------------ #


with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print("AUC", compute_auc(pos_score, neg_score))


# Thumbnail credits: Link Prediction with Neo4j, Mark Needham
# sphinx_gallery_thumbnail_path = '_static/blitz_4_link_predict.png'

In epoch 0, loss: 0.7010293006896973
In epoch 5, loss: 0.6924521923065186
In epoch 10, loss: 0.6872186660766602
In epoch 15, loss: 0.674706757068634
In epoch 20, loss: 0.6486778259277344
In epoch 25, loss: 0.6004092693328857
In epoch 30, loss: 0.5699384808540344
In epoch 35, loss: 0.5552774667739868
In epoch 40, loss: 0.5337749123573303
In epoch 45, loss: 0.5180059671401978
In epoch 50, loss: 0.500512957572937
In epoch 55, loss: 0.4778256416320801
In epoch 60, loss: 0.44953662157058716
In epoch 65, loss: 0.40838900208473206
In epoch 70, loss: 0.3526500165462494
In epoch 75, loss: 0.2958819568157196
In epoch 80, loss: 0.2588243782520294
In epoch 85, loss: 0.23327195644378662
In epoch 90, loss: 0.2113390564918518
In epoch 95, loss: 0.19299021363258362
AUC 0.8264046180454167
